### Improving Epidemiological Diseases Mitigation: A case study of Monkeypox using Agent-based Modelling and Machine Learning Approaches.

#### STAGE 3: Continous cleaning, Encode categorical variables into numeri, dataset train,test splitting, Models implementation.

#### At this stage I also eccounter difficulty of getting different result from RF, XGBoost, DT, and Neural Network. Resolve by setting RANDOM_STATE seed to 42.

In [3]:
#!pip install xgboost

In [82]:
# import required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [83]:
# imports the Python warnings module to provides a mechanism for handling warnings that occur during program execution.
import warnings
warnings.filterwarnings('ignore')

In [84]:
# Load first clean dataset
df_mkp = pd.read_csv('first_cleaned_dataset.csv')
df_mkp.head(5)


,Status,Location,City,Country,Age,Gender,Date_onset,Date_confirmation,Symptoms,Hospitalised (Y/N/NA),...,Contact_comment,Contact_location,Travel_history (Y/N/NA),Travel_history_entry,Travel_history_start,Travel_history_location,Travel_history_country,Genomics_Metadata,Date_entry,difference
0,confirmed,Guy's and St Thomas Hospital London,London,England,20-64,male,2022-04-29,2022-06-05,rash,Y,...,Under investigation,Household,Y,04/05/2022,late April,Lagos and Delta States,Nigeria,West African Clade,18/05/2022,37 days
1,confirmed,Guy's and St Thomas Hospital London,London,England,20-64,male,2022-05-05,2022-12-05,rash,Y,...,Index Case of household cluster,Household,N,16/05/2022,late April,Europe,Spain,West African Clade,18/05/2022,214 days
2,confirmed,London,London,England,20-64,male,2022-04-30,2022-05-13,vesicular rash,N,...,Under investigation,Household,N,16/05/2022,late April,Europe,Spain,West African Clade,18/05/2022,13 days
3,confirmed,London,London,England,20-64,male,2022-04-29,2022-05-15,vesicular rash,Y,...,Under investigation,Household,N,16/05/2022,late April,Europe,Spain,West African Clade,18/05/2022,16 days
4,confirmed,London,London,England,20-64,male,2022-04-29,2022-05-15,vesicular rash,Y,...,Under investigation,Household,N,16/05/2022,late April,Europe,Spain,West African Clade,18/05/2022,16 days


In [85]:
#Check dataset shape: number of rows and columns
df_mkp.shape

(1132, 24)

In [86]:
#check for missing value
df_mkp.isna().sum()

Status                     0
Location                   0
City                       0
Country                    0
Age                        0
Gender                     0
Date_onset                 0
Date_confirmation          0
Symptoms                   0
Hospitalised (Y/N/NA)      0
Date_hospitalisation       0
Isolated (Y/N/NA)          0
Date_isolation             0
Outcome                    0
Contact_comment            0
Contact_location           0
Travel_history (Y/N/NA)    0
Travel_history_entry       0
Travel_history_start       0
Travel_history_location    0
Travel_history_country     0
Genomics_Metadata          0
Date_entry                 0
difference                 0
dtype: int64

## Dataset Second Preprocessing 

In [88]:
# Check to confirm removal of error data
df_mkp[df_mkp['Country'] == 'Spain'].groupby('City')['Status'].count()

City
Almeria            3
Aragon             1
Barcelona          1
Cadiz              3
Costa del Sol      1
Fuerteventura      2
Gran Canaria      16
Madrid           275
Malaga            16
Seville            2
Tenerife           4
Valencia           2
Valladolid         1
Name: Status, dtype: int64

In [89]:
#Check dataset shape: number of rows and columns
df_mkp.shape

(1132, 24)

In [90]:
# Encode categorical variables into numeric for better model impllementation
categorical_features = df_mkp.select_dtypes(include=["object"]).columns.tolist()
for feature in categorical_features:
    le = LabelEncoder()
    df_mkp[feature] = le.fit_transform(df_mkp[feature])

In [91]:
# Display dataset
df_mkp.head(10)

,Status,Location,City,Country,Age,Gender,Date_onset,Date_confirmation,Symptoms,Hospitalised (Y/N/NA),...,Contact_comment,Contact_location,Travel_history (Y/N/NA),Travel_history_entry,Travel_history_start,Travel_history_location,Travel_history_country,Genomics_Metadata,Date_entry,difference
0,0,47,42,12,3,1,0,20,23,1,...,12,0,1,0,2,6,10,1,5,24
1,0,47,42,12,3,1,2,21,23,1,...,8,0,0,3,2,5,13,1,5,13
2,0,59,42,12,3,1,1,4,31,0,...,12,0,0,3,2,5,13,1,5,7
3,0,59,42,12,3,1,0,5,31,1,...,12,0,0,3,2,5,13,1,5,8
4,0,59,42,12,3,1,0,5,31,1,...,12,0,0,3,2,5,13,1,5,8
5,0,59,42,12,3,1,0,5,31,0,...,12,0,0,3,2,5,13,1,5,8
6,0,75,54,12,3,1,0,5,31,1,...,12,0,1,3,2,8,5,1,5,8
7,0,55,39,36,2,1,0,6,28,0,...,12,0,0,3,2,5,13,1,5,9
8,0,55,39,36,2,1,0,6,28,0,...,12,0,0,3,2,5,13,1,5,9
9,0,55,39,36,2,1,0,6,28,0,...,12,0,0,3,2,5,13,1,5,9


In [92]:
# Check number of columns and rows
df_mkp.shape

(1132, 24)

In [93]:
# Drop unwanted features from the DataFrame
df_mkp.drop("difference", axis=1, inplace=True)

In [94]:
# Check number of columns and rows
df_mkp.shape

(1132, 23)

In [95]:
# Display  first 20 dataset from backward
df_mkp.tail(20)

,Status,Location,City,Country,Age,Gender,Date_onset,Date_confirmation,Symptoms,Hospitalised (Y/N/NA),...,Outcome,Contact_comment,Contact_location,Travel_history (Y/N/NA),Travel_history_entry,Travel_history_start,Travel_history_location,Travel_history_country,Genomics_Metadata,Date_entry
1112,0,88,45,6,3,1,0,1,15,0,...,0,12,0,1,3,2,5,13,1,3
1113,0,88,45,6,3,1,0,1,15,0,...,0,12,0,1,3,2,5,13,1,3
1114,0,88,45,6,3,1,0,1,15,0,...,0,12,0,1,3,2,5,13,1,3
1115,0,88,45,6,3,1,0,1,15,0,...,0,12,0,1,3,2,5,13,1,3
1116,0,88,45,6,3,1,0,1,15,0,...,0,12,0,1,3,2,5,13,1,3
1117,0,88,45,6,3,1,0,1,15,0,...,0,12,0,1,3,2,5,13,1,3
1118,0,88,45,6,3,1,0,1,15,0,...,0,12,0,1,3,2,5,13,1,3
1119,0,88,45,6,3,1,0,1,15,0,...,0,12,0,1,3,2,5,13,1,3
1120,0,88,45,6,3,1,0,1,15,0,...,0,12,0,1,3,2,5,13,1,3
1121,0,88,45,6,3,1,0,1,15,0,...,0,12,0,1,3,2,5,13,1,3


In [96]:
# Save second cleaned dataset
#df_mkp.to_csv('second_cleaned_dataset.csv', index=False)
#print("Cleaned dataset save to the directory successfully!")

Cleaned dataset save to the directory successfully!


In [97]:
# Count Status by category
df_mkp['Status'].value_counts()

0    920
1    145
2     67
Name: Status, dtype: int64

### Required SMOTE method to balance the dataset for the modeles to learn across all classes

In [19]:
#!pip install --upgrade scikit-learn
#!pip install imbalanced-learn
#!pip install scikit-learn==0.24.2
#!pip install imbalanced-learn==0.8.0

In [98]:
# Import libraries
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

In [99]:
# Separate features and target variable
X = df_mkp.drop('Status', axis=1)
y = df_mkp['Status']

In [100]:
# Initialize SMOTE (Synthetic Minority Over-sampling Technique)
smote = SMOTE(random_state=42)

In [101]:
# Apply SMOTE to balance the dataset
X_resampled, y_resampled = smote.fit_resample(X, y)

In [102]:
# Combine the resampled features and target variable into a new DataFrame
df_resampled = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.Series(y_resampled, name='Status')], axis=1)

In [103]:
# Check the distribution after balancing
print(df_resampled['Status'].value_counts())

0    920
1    920
2    920
Name: Status, dtype: int64


In [104]:
# Split data
# Separate features and target variable
X = df_mkp.drop('Status', axis=1)
y = df_mkp['Status']

#convert y back to Dataframe
y=y.to_frame()
 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [105]:
y_train

,Status
12,0
381,1
717,0
497,0
964,0
...,...
121,0
1044,0
1095,0
860,0


In [106]:
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (905, 22)
X_test shape: (227, 22)
y_train shape: (905, 1)
y_test shape: (227, 1)


## Model Implementation and Evaluation

In [108]:
# Model Implementation and Evaluation
models = {}

In [109]:
# Support Vector Machine (SVM)
svm = SVC(kernel="linear", probability=True)  # Enable probability estimates
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)

# Use predict_proba for obtaining class probabilities
y_pred_svm_probs = svm.predict_proba(X_test)

svm_metrics = {
    "Accuracy": accuracy_score(y_test, y_pred_svm),
    "Precision": precision_score(y_test, y_pred_svm, average='weighted'),
    "Recall": recall_score(y_test, y_pred_svm, average='weighted'),
    "F1-score": f1_score(y_test, y_pred_svm, average='weighted'),
    "AUC-ROC": roc_auc_score(y_test, y_pred_svm_probs, multi_class='ovr'),
}

In [110]:
print(svm_metrics)

{'Accuracy': 0.8193832599118943, 'Precision': 0.7997554994796343, 'Recall': 0.8193832599118943, 'F1-score': 0.7892200717982402, 'AUC-ROC': 0.8950499757517965}


In [113]:

# Random Forest (RF)
# Define a random state to maintain same result at every run
RANDOM_STATE = 42

# Random Forest (RF)
rf = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

# Store metrics directly
models["RF"] = {
    "metrics": {
        "Accuracy": accuracy_score(y_test, y_pred_rf),
        "Precision": precision_score(y_test, y_pred_rf, average='weighted'),
        "Recall": recall_score(y_test, y_pred_rf, average='weighted'),
        "F1-score": f1_score(y_test, y_pred_rf, average='weighted'),
        "AUC-ROC": roc_auc_score(y_test, rf.predict_proba(X_test), multi_class='ovr'),
    },
    "model": rf  # Optionally store the model itself 
}

# Output the stored metrics
print(models["RF"])

{'metrics': {'Accuracy': 0.9559471365638766, 'Precision': 0.9598535001778671, 'Recall': 0.9559471365638766, 'F1-score': 0.9574727538081215, 'AUC-ROC': 0.9895691139835456}, 'model': RandomForestClassifier(random_state=42)}


In [115]:
# Decision Tree (DT)
# Add a random state for reproducibility
RANDOM_STATE = 42

# Decision Tree (DT)
dt = DecisionTreeClassifier(random_state=RANDOM_STATE)
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)

models["DecisionTree"] = {
    "metrics": {
        "Accuracy": accuracy_score(y_test, y_pred_dt),
        "Precision": precision_score(y_test, y_pred_dt, average='weighted'),
        "Recall": recall_score(y_test, y_pred_dt, average='weighted'),
        "F1-score": f1_score(y_test, y_pred_dt, average='weighted'),
        "AUC-ROC": roc_auc_score(y_test, dt.predict_proba(X_test), multi_class='ovr'),
    },
    "model": dt  # Optionally store the model itself 
}

# Print the metrics
print(models["DecisionTree"])

{'metrics': {'Accuracy': 0.9515418502202643, 'Precision': 0.9569500159968498, 'Recall': 0.9515418502202643, 'F1-score': 0.9536912935051284, 'AUC-ROC': 0.952449081180568}, 'model': DecisionTreeClassifier(random_state=42)}


In [117]:
# XGBoost
xgb_model = xgb.XGBClassifier()
models["XGBoost"] = xgb_model
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)
models["XGBoost"] = {
    "Accuracy": accuracy_score(y_test, y_pred_xgb),
    "Precision": precision_score(y_test, y_pred_xgb, average='weighted'),
    "Recall": recall_score(y_test, y_pred_xgb, average='weighted'),
    "F1-score": f1_score(y_test, y_pred_xgb, average='weighted'),
    "AUC-ROC": roc_auc_score(y_test, xgb_model.predict_proba(X_test), multi_class='ovr'),
}

print(models["XGBoost"])

{'Accuracy': 0.9427312775330396, 'Precision': 0.9538742251673064, 'Recall': 0.9427312775330396, 'F1-score': 0.9466552843436243, 'AUC-ROC': 0.990064870726659}


In [118]:
# import require library

import tensorflow as tf
from keras.utils import to_categorical
import random
import os

# Set the random seeds
RANDOM_STATE = 42
os.environ['PYTHONHASHSEED']=str(RANDOM_STATE)
random.seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)

# Neural Network (Multiclass)
model_nn = Sequential()
model_nn.add(Dense(128, activation="relu", input_shape=(X_train.shape[1],)))
model_nn.add(Dense(64, activation="relu"))
model_nn.add(Dense(len(df_mkp['Status'].unique()), activation="softmax"))  # Adjust the number of classes
model_nn.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# One-hot encode the target variable
y_train_one_hot = to_categorical(y_train)

# Train the model
model_nn.fit(X_train, y_train_one_hot, epochs=10, batch_size=32)

# Predictions
y_pred_nn = model_nn.predict(X_test)

# Metrics
metrics_nn = {
    "Accuracy": accuracy_score(y_test, np.argmax(y_pred_nn, axis=1)),
    "Precision": precision_score(y_test, np.argmax(y_pred_nn, axis=1), average='weighted'),
    "Recall": recall_score(y_test, np.argmax(y_pred_nn, axis=1), average='weighted'),
    "F1-score": f1_score(y_test, np.argmax(y_pred_nn, axis=1), average='weighted'),
    "AUC-ROC": roc_auc_score(y_test, y_pred_nn, multi_class='ovr'),
}

print("Neural Network Metrics:", metrics_nn)

Epoch 1/10
29/29 [==============================] - 2s 3ms/step - loss: 0.8810 - accuracy: 0.7492
Epoch 2/10
29/29 [==============================] - 0s 3ms/step - loss: 0.4121 - accuracy: 0.8442
Epoch 3/10
29/29 [==============================] - 0s 3ms/step - loss: 0.3329 - accuracy: 0.8475
Epoch 4/10
29/29 [==============================] - 0s 3ms/step - loss: 0.3319 - accuracy: 0.8541
Epoch 5/10
29/29 [==============================] - 0s 3ms/step - loss: 0.3357 - accuracy: 0.8541
Epoch 6/10
29/29 [==============================] - 0s 3ms/step - loss: 0.3049 - accuracy: 0.8674
Epoch 7/10
29/29 [==============================] - 0s 3ms/step - loss: 0.3093 - accuracy: 0.8674
Epoch 8/10
29/29 [==============================] - 0s 3ms/step - loss: 0.2716 - accuracy: 0.8862
Epoch 9/10
29/29 [==============================] - 0s 3ms/step - loss: 0.2821 - accuracy: 0.8619
Epoch 10/10
8/8 [==============================] - 0s 1ms/step
Neural Network Metrics: {'Accuracy': 0.8546255506607929